In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#/content/drive/MyDrive

In [ ]:
project_id = 'tweetprediction'
bucket_name = 'ticks_with_indicators_with_volume'
bucket_name_tick_data = 'bittrex_tick_data'

In [ ]:
#from google.colab import auth
#auth.authenticate_user()

In [ ]:
# try:
#   import talib
# except:
#   #!cp /content/drive/MyDrive/Privat/Crypto/ta-lib-0.4.0-src.tar.gz ta-lib-0.4.0-src.tar.gz
#   !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#   !tar -xzvf ta-lib-0.4.0-src.tar.gz
#   # !unzip /content/drive/MyDrive/Privat/Crypto/talibmaked.zip -d /
#   %cd ta-lib
#   !./configure --prefix=/usr
#   !make
#   !make install
#   !pip install Ta-Lib
  
#   import talib
#   from talib import MA_Type

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import json
import copy
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from datetime import datetime

### Add custom import paths for DataStreamCreator and IndicatorCalculator

In [ ]:
# Get the current directory
current_dir = os.getcwd()
current_dir_splitted = current_dir.split(os.sep)
current_dir_splitted

In [ ]:
# Create the import directories for the DataStreamCreator and the IndicatorCalculator
dsc_dir = os.path.join(os.sep, *current_dir_splitted[:-2], 'DataStreamCreator')
print(f"dsc_dir: {dsc_dir}")

ind_dir = os.path.join(os.sep, *current_dir_splitted[:-2], 'IndicatorCalculator')
print(f"ind_dir: {ind_dir}")

# Add them to the import paths
sys.path.insert(0, dsc_dir)
sys.path.insert(0, ind_dir)

In [ ]:
# Import the actual classes
from IndicatorCalculator import IndicatorCalculator
import DataStreamCreator

In [ ]:
import tensorflow as tf

In [ ]:
# from tensorflow.keras.layers import Input, Dense, ReLU, Add, Flatten, Concatenate, LayerNormalization, UpSampling2D, Activation, LSTM, Multiply, Dropout, Reshape, Permute, BatchNormalization, MaxPooling1D, AveragePooling1D, MaxPooling3D, AveragePooling2D, LayerNormalization, MaxPooling2D
# from tensorflow.keras.layers import Conv1D, Conv2D
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# Define all the parameters and variables

In [ ]:
# Define the data path
DATA_PATH = os.path.join(os.sep, *current_dir_splitted[:-2], 'DemoData')
print(f"DATA_PATH: {DATA_PATH}")

In [ ]:
# Define a global random seed
RANDOM_SEED = 11

In [ ]:
# checkpoint_path = "gs://ticks_with_indicators_with_volume/chk/"

In [ ]:
# Load the example OHLCV file
EXAMPLE_FILE_PATH = os.path.join(DATA_PATH, "BTC-USDT.csv")
EXAMPLE_FILE_PATH

In [ ]:
# BATCH_SIZE = 64
X_BLOCK_LENGHT = 128

In [ ]:
DATASET_NUMBER = 5

In [ ]:
# Parameterset 7
PARAM_SET = 7

Y_TYPE = 3 # 0 for categorical, 1 for float, 2 for gain lookaround, 3 for entry/exit
SMCNT = 48
SMCNT2 = 48

EXPECTED_GAIN_LOOKFORWARD = 1*24
ENTR_THR = 0.9
ENTR_THR2 = 0.8
ENTR_THR3 = 0.0
EXIT_THR = -0.5
EXIT_THR2 = 0.1

Y_LOOKAHEAD_CNT = 1
GAIN_LOOKAROUND_CNT = 1

CLASS_WEIGHT = None

In [ ]:
SHORTSPAN = 6
MIDSPAN = 48
LONGSPAN = 120

In [ ]:
BOS_TOKEN_ID = 255
PAD_TOKEN_ID = 255
EOS_TOKEN_ID = 255
MASK_TOKEN_ID = 255

MASK_FACTOR = 0.15

In [ ]:
# Set the seed for token masking random
np.random.seed(RANDOM_SEED)

In [ ]:
import tensorflow as tf
from tensorflow.python.lib.io import file_io

#

# Demonstration of XBlockGenerator

The `XBlockGenerator` class is used to generate time-frame slices (= X-blocks) out of a time series table of tick and indicator data.
For this task, an input `pd.DataFrame` `tickAndIndicatorDF` is processed row by row.
It is called 'X' because its purpose is to be used as input data for machine learning networks (== X-data).

Every X-block is created by using a specific amount of table rows, defined by the `int` parameter `X_Block_lenght`.
If the tick data is in hours and `X_lookback_cnt=12`, the generator would return slices of 12 hours.
The step size between the X-blocks is 1, so the resulting DFs in the example would be: 00:00-11:00, 01:00-12:00, 02:00-13:00, ...

For demonstration purposes, only a OHLCV dataframe without indicators is used, to keep the confusion low!

In [ ]:
# Load the OHLCV table
tickdata = pd.read_csv(EXAMPLE_FILE_PATH)
tickdata.set_index("startsAt", inplace = True) # Todo: Take care of the index name
tickdata.sort_index()
 
# Crop the lookback
# Todo: Describe why cropping is necessary
# tickdata = tickdata.iloc[X_BLOCK_LENGHT:]

tickdata

In [ ]:
# Plot the open column of the data set
fig, ax1 = plt.subplots(figsize=(20,10))

ax1.plot(tickdata.loc[:,'open'], color="black")

In [ ]:
# Generate 2 X-Blocks with a size of 100 timesteps
xBlockGenerator = DataStreamCreator.XBlockGenerator(tick_and_indicator_DF=tickdata,
                                                    generator_batch_size=2,
                                                    X_Block_lenght=100
                                                    )
xBlocks = next(xBlockGenerator)
print(f"tickdata.shape: {tickdata.shape}")
print(f"xBlocks.shape: {xBlocks.shape}")

In [ ]:
# The generated X-blocks do not have column names any more, as they are just an array
# Plot the first feature index of the dataset (which is the open price)
# The gray line is the 'whole' dataset
# The red one is the slice of the X-Block

fig, ax1 = plt.subplots(figsize=(20,10))

ax1.plot(tickdata.loc[:,'open'].values[:500], color="lightgray")

# Plot the first block
ax1.plot(xBlocks[0,:,0], color="red", linewidth = 4)

In [ ]:
# Here, the second block is plotted. It is shifted against the "base data", as
# it's first datapoint is at timestamp 1. But it also reached one more timestamp
# into the future.
# Shifting is because X-blocks are simple array without "global" time information,
# as they are just an array

fig, ax1 = plt.subplots(figsize=(20,10))

ax1.plot(tickdata.loc[:,'open'].values[:200], color="lightgray")

# Plot the first block
ax1.plot(xBlocks[0,:,0], color="red", linewidth = 4)

# Plot the second block
ax1.plot(xBlocks[1,:,0], color="green", linewidth = 4)

In [ ]:
# Generate X-Blocks until there is no more data to get the block
# at the end of the dataset

latest_blocks = None

while True:
  try:
    latest_blocks = next(xBlockGenerator)
  except StopIteration:
    # Exit if there is no more data
    break
latest_blocks.shape

In [ ]:
# Plot the latest data
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.plot(latest_blocks[-1,:,0], color="red", linewidth = 4)

In [ ]:
# Plot the end of the data table
fig, ax1 = plt.subplots(figsize=(20,10))

ax1.plot(tickdata.loc[:,'open'].values[-500:], color="lightgray")

# Demonstration of YDataGenerator

The `YDataGenerator` class is used to generate future information for training out of a time series table of tick and indicator data.
This can for example be the assets relative price or gain in 24 hours, the direction of movement, or trade entry and exit signals.

IMPORTANT: This class can of course not look into the future, so if you want to output the price in 24 hours, this can only be done until
table index `len(timestamp rows) - 24`, otherwise it would really have to look into the future!

The purpose of this class is to generate machine-learning target data (y-values) according to the X-Blocks generated by the `XBlockGenerator`.
The X-Block contains data from the past, which is known in a live application, while the Y data is unknown in the live application, and therefore has to be predicted.
Here, it can be generated for training purposes using historical data.

As a price basis for calculating the y data, the `open` column of the input table `tick_DF` is used.

There are 4 different types of y data available, which are shown each on its own in the following section.

---
### Visualization of y for Y_DATA_TYPE_DIRECTION_FLOAT

In [ ]:
# Get the template parameter dict for this y type
Y_PARAM_DICT_TEMPLATE = DataStreamCreator.YDataGenerator.PARAM_DICT_TEMPLATE_Y_DATA_TYPE_DIRECTION_FLOAT
Y_PARAM_DICT_TEMPLATE

In [ ]:
yDataGenerator = DataStreamCreator.YDataGenerator(tick_DF=tickdata,
                                                    generator_batch_size=2,
                                                    lookback_cnt=100,
                                                    y_type_dict=Y_PARAM_DICT_TEMPLATE
                                                    )
yData = next(yBlockGenerator)
print(f"yData.shape: {yData.shape}")

In [ ]:
y_dir = None
y_dir2nd = None

ite = DataStreamCreator.FileListToDataStream([testfiles[VISUAL_ID]], 31, TEST_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              expected_gain_lookforward=EXPECTED_GAIN_LOOKFORWARD, entr_thr=ENTR_THR, entr_thr2=ENTR_THR2, entr_thr3=ENTR_THR3,
                                              exit_thr=EXIT_THR, exit_thr2=EXIT_THR2,
                                              parallel_generators = 1, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              shuffle = False)
while True:
  try:    
    ne = next(ite)
  except StopIteration:
    break
  if y_dir is None or y_dir2nd is None:
    y_dir = ne[1][:,0]
    y_dir2nd = ne[1][:,1]
  else:
    y_dir = np.concatenate([y_dir, ne[1][:,0]])
    y_dir2nd = np.concatenate([y_dir2nd, ne[1][:,1]])

In [ ]:
# Load the tick table
tickpath = os.path.join(TEST_PATH, testfiles[VISUAL_ID])
tickdata = pd.read_csv(tickpath)
tickdata.set_index("startsAt", inplace = True)
tickdata.sort_index()
 
# Crop the lookback
tickdata = tickdata.iloc[X_LOOKBACK_CNT:]

tickdata

In [ ]:
y_dir.shape

In [ ]:
fig, ax1 = plt.subplots(figsize=(20,10))

ax1.plot(y_dir2nd, color="red")
ax1.plot(y_dir, color="green")

# ax1.set_xlim(0,1000)

ax2 = ax1.twinx()

y_dir_naned = copy.deepcopy(y_dir)
y_dir_naned[y_dir_naned == 0] = np.nan

exit_naned = copy.deepcopy(y_dir2nd)
exit_naned[exit_naned == 0] = np.nan

ax2.plot(tickdata.loc[:,"close"].values, color = "black")

ax1.set_ylim(-1.0,1.0)

# Visualization of y for Y_DATA_TYPE_DIRECTION_CATEGORICAL





In [ ]:
VISUAL_ID = 1

In [ ]:
Y_TYPE = DataStreamCreator.YDataGenerator.Y_DATA_TYPE_DIRECTION_CATEGORICAL

In [ ]:
GAIN_LOOKAROUND_CNT = 24

In [ ]:
y_category = None

ite = DataStreamCreator.FileListToDataStream([testfiles[VISUAL_ID]], 31, TEST_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              expected_gain_lookforward=EXPECTED_GAIN_LOOKFORWARD, entr_thr=ENTR_THR, entr_thr2=ENTR_THR2, entr_thr3=ENTR_THR3,
                                              exit_thr=EXIT_THR, exit_thr2=EXIT_THR2,
                                              parallel_generators = 1, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              shuffle = False)
while True:
  try:    
    ne = next(ite)
  except StopIteration:
    break
  if y_category is None:
    y_category = ne[1]
  else:
    y_category = np.concatenate([y_category, ne[1]])

In [ ]:
# Load the tick table
tickpath = os.path.join(TEST_PATH, testfiles[VISUAL_ID])
tickdata = pd.read_csv(tickpath)
tickdata.set_index("startsAt", inplace = True)
tickdata.sort_index()
 
# Crop the lookback
tickdata = tickdata.iloc[X_LOOKBACK_CNT:]

tickdata

In [ ]:
y_category.shape

In [ ]:
# This chart shows the categories (0=falling, 1=neutral, 2=rising)
fig, ax1 = plt.subplots(figsize=(20,10))

ax1.plot(y_category, color="blue", linewidth=3)

ax2 = ax1.twinx()

ax2.plot(tickdata.loc[:,"close"].values, color = "black")

In [ ]:
# This chart shows a comparison of the categories to the Y_DATA_TYPE_DIRECTION_FLOAT values
fig, ax1 = plt.subplots(figsize=(20,10))

ax1.plot(y_dir2nd, color="red", linestyle=":") # Data from the previous example (Y_DATA_TYPE_DIRECTION_FLOAT)
ax1.plot(y_dir, color="green", linestyle=":") # Data from the previous example (Y_DATA_TYPE_DIRECTION_FLOAT)
ax1.plot(y_category, color="blue", linewidth=3)

# ax1.set_xlim(0,1000)

ax2 = ax1.twinx()

ax2.plot(tickdata.loc[:,"close"].values, color = "black")

# Visualization of y for Y_DATA_TYPE_GAIN_LOOKAROUND


In [ ]:
VISUAL_ID = 1

In [ ]:
Y_TYPE = 2 #DataStreamCreator.YDataGenerator.Y_DATA_TYPE_DIRECTION_CATEGORICAL

In [ ]:
y_gains = None
# gains[:, 0] = np.tanh(_max_past_gain_slice)
# gains[:, 1] = np.tanh(_max_past_gain_ma_slice)
# gains[:, 2] = np.tanh(_max_past_gain_dir_slice)

# gains[:, 3] = np.tanh(_max_future_gain_slice)
# gains[:, 4] = np.tanh(_max_future_gain_ma_slice)
# gains[:, 5] = np.tanh(_max_future_gain_dir_slice)

ite = DataStreamCreator.FileListToDataStream([testfiles[VISUAL_ID]], 31, TEST_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              expected_gain_lookforward=EXPECTED_GAIN_LOOKFORWARD, entr_thr=ENTR_THR, entr_thr2=ENTR_THR2, entr_thr3=ENTR_THR3,
                                              exit_thr=EXIT_THR, exit_thr2=EXIT_THR2,
                                              parallel_generators = 1, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              shuffle = False)
while True:
  try:    
    ne = next(ite)
  except StopIteration:
    break
  if y_gains is None:
    y_gains = ne[1]
  else:
    y_gains = np.concatenate([y_gains, ne[1]])

In [ ]:
# Load the tick table
tickpath = os.path.join(TEST_PATH, testfiles[VISUAL_ID])
tickdata = pd.read_csv(tickpath)
tickdata.set_index("startsAt", inplace = True)
tickdata.sort_index()
 
# Crop the lookback
tickdata = tickdata.iloc[X_LOOKBACK_CNT:]

tickdata.shape

In [ ]:
# Todo important: Not the same size as the tick table!!!!
y_gains.shape

In [ ]:
# This chart shows a comparison of the categories to the Y_DATA_TYPE_DIRECTION_FLOAT values
fig, ax1 = plt.subplots(figsize=(20,10))

ax1.plot(y_gains[:,0], color="lightgray")
ax1.plot(y_gains[:,1], color="red", label="Past gain")
ax1.plot(y_gains[:,4], color="green", label="Future gain")

# ax1.set_xlim(0,1000)

ax2 = ax1.twinx()

ax2.plot(tickdata.loc[:,"close"].values, color = "black")

In [ ]:
# This chart shows a comparison of the categories to the Y_DATA_TYPE_DIRECTION_FLOAT values
fig, ax1 = plt.subplots(figsize=(20,10))

ax1.plot(y_gains[:,2], color="red", label="Past gain derviation")
ax1.plot(y_gains[:,5], color="green", label="Future gain derviation")

# ax1.set_xlim(0,1000)

ax2 = ax1.twinx()

ax2.plot(tickdata.loc[:,"close"].values, color = "black")

In [ ]:
stop

# Testing

In [ ]:
import pickle
from sklearn.cluster import Birch

In [ ]:
# Load the BIRCH categorizer
with open("/content/drive/MyDrive/Privat/Crypto/birchNew.pkl", 'rb') as pickle_file:
  brc = pickle.load(pickle_file)

In [ ]:
def NormForBirch(XIn):
  _THR = 100
  normed = copy.deepcopy(XIn)

  normed[normed >= _THR] = _THR
  normed[normed <= -_THR] = -_THR

  return normed

In [ ]:
def XFrameGenerator(oneTestFileName):

  ite = DataStreamCreator.FileListToDataStream([oneTestFileName], BATCH_SIZE, TEST_PATH,
                                                smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                                X_lookback_cnt = X_LOOKBACK_CNT,
                                                y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                                gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                                shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                                y_type = Y_TYPE,
                                                expected_gain_lookforward=EXPECTED_GAIN_LOOKFORWARD, entr_thr=ENTR_THR, entr_thr2=ENTR_THR2, entr_thr3=ENTR_THR3,
                                                exit_thr=EXIT_THR, exit_thr2=EXIT_THR2,
                                                parallel_generators = 1, random_seed=RANDOM_SEED,
                                                y_exponent = 1.0,
                                                shuffle = False)
  while True:
    try:    
      ne = next(ite)

      # Xin as in the mask function
      Xin = ne[0]

      # y can be used directly
      y = ne[1]

      # Store the input shape
      inputShape = Xin.shape

      # Create a "sentence"
      X_normed = NormForBirch(Xin)
      X_normed = np.reshape(X_normed, (X_normed.shape[0] * X_normed.shape[1], X_normed.shape[2]))
      sentence = brc.predict(X_normed)
      sentence = np.reshape(sentence, inputShape[:2])

      yield sentence
    
    except StopIteration:
      break
    
xFrameGenDebug = XFrameGenerator(testfiles[VISUAL_ID])
xFrameGenDebug

In [ ]:
ne = next(xFrameGenDebug)

In [ ]:
ne.shape

In [ ]:
ne[0,:]

# Load the model

In [ ]:
cfg = RobertaConfig()
cfg.vocab_size = 256
cfg.max_position_embeddings = X_LOOKBACK_CNT
cfg.bos_token_id = BOS_TOKEN_ID
cfg.pad_token_id = PAD_TOKEN_ID
cfg.eos_token_id = EOS_TOKEN_ID
cfg

In [ ]:
#@title CreateModelRobertaEntryExit
def CreateModelRobertaEntryExit():

  # Build your model input
  inputSentence = Input(shape=(X_LOOKBACK_CNT), name='input', dtype='int32')
  
  nlp = TFRobertaModel(cfg, name="Roberta")(inputSentence)
  flat = Flatten(name="FlattenNLP")(nlp['last_hidden_state'])

  categories = Dense(3, name="DenseCategories", activation="softmax")(flat)

  output = Concatenate(name="ConcatOutput")([categories])
  outputs = [output]

  mnamesuffix = "_1"

  # And combine it all in a model object
  model = Model(inputs=inputSentence, outputs=outputs, name='ModelRobertaEntryExit'+mnamesuffix)

  return model

model = CreateModelRobertaEntryExit()
model.summary(line_length=220)

In [ ]:
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-06,
    beta_1=0.9,
    beta_2=0.98,
    decay=0.01)

In [ ]:
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = tf.keras.losses.MeanSquaredError(), 
    metrics=None)

In [ ]:
copy_filenames = ['gs://ticks_with_indicators_with_volume/chk/TPU_ModelRobertaEntryExit_1_128LB_256VC_FullPrediction/cp_daily_valid_27_end/model.h5'
                  ]
for p in copy_filenames:
  fn = p.split("/")[-1]
  cpnt = p.split("/")[-2]

  os.mkdir(os.path.join("/content", cpnt))

  localPath = os.path.join("/content", cpnt, fn)

  if ("model.h5" in p):
    localPathModel = localPath
  elif ("w.pickle" in p):
    localPathW = localPath

  with file_io.FileIO(p, mode='rb') as input_f:
    with file_io.FileIO(localPath, mode='wb+') as output_f:
      output_f.write(input_f.read())
      print("Pulled from bucket: '" + fn + "'")

In [ ]:
print(f"Loading {localPathModel}")
model.load_weights(localPathModel)

In [ ]:
xFrameGenDebug = XFrameGenerator(testfiles[VISUAL_ID]) # Todo: Return shape in gen
p = model.predict(xFrameGenDebug)
p.shape

In [ ]:
# Adapt prediction to input shape (a window in the beginning cannot be predicted, as lookback into the past is required)
# Todo: Get shape from gen
pFullsize = np.empty((y_dir.shape[0], p.shape[1]))
pFullsize[:] = np.nan

pFullsize[-p.shape[0]:,:] = p

In [ ]:
y_dir.shape

In [ ]:
pFullsize.shape

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))

# ax1.plot(y_dir2nd, color="red")
ax1.plot(y_dir, color="gray")
ax1.plot(pFullsize[:,0], color="green")

# ax1.set_xlim(0,1000)

ax2 = ax1.twinx()

y_dir_naned = copy.deepcopy(y_dir)
y_dir_naned[y_dir_naned == 0] = np.nan

exit_naned = copy.deepcopy(y_dir2nd)
exit_naned[exit_naned == 0] = np.nan

ax2.plot(tickdata.loc[:,"close"].values, color = "black")

ax1.set_ylim(-1.1,1.1)

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))

# ax1.plot(y_dir2nd, color="gray")
ax1.plot(pFullsize[:,1], color="red")

# ax1.set_xlim(0,1000)

ax2 = ax1.twinx()

y_dir_naned = copy.deepcopy(y_dir)
y_dir_naned[y_dir_naned == 0] = np.nan

exit_naned = copy.deepcopy(y_dir2nd)
exit_naned[exit_naned == 0] = np.nan

ax2.plot(tickdata.loc[:,"close"].values, color = "black")

# ax1.set_ylim(-1.1,1.1)

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))

# ax1.plot(y_dir2nd, color="gray")
ax1.plot(pFullsize[:,0], color="green")
ax1.plot(pFullsize[:,1], color="red")

# ax1.set_xlim(0,1000)

ax2 = ax1.twinx()

y_dir_naned = copy.deepcopy(y_dir)
y_dir_naned[y_dir_naned == 0] = np.nan

exit_naned = copy.deepcopy(y_dir2nd)
exit_naned[exit_naned == 0] = np.nan

ax2.plot(tickdata.loc[:,"close"].values, color = "black")

ax1.set_ylim(0,1.1)

In [ ]:
ENTRY_THR = 0.4

fig, ax1 = plt.subplots(figsize=(30,10))

# ax1.plot(y_dir2nd, color="red")
# ax1.plot(y_dir, color="gray")
ax1.plot(pFullsize[:,0], color="gray")

# ax1.set_xlim(0,1000)

ax2 = ax1.twinx()

y_dir_naned = copy.deepcopy(y_dir)
y_dir_naned[y_dir_naned == 0] = np.nan

entries = np.array([pFullsize[:,0] >= ENTRY_THR])[0,:] * tickdata.loc[:,"close"].values
entries[entries == 0] = np.nan

ax2.plot(tickdata.loc[:,"close"].values, color = "black")
ax2.plot(entries, color = "green", marker="X", markersize=10)

ax1.set_ylim(0,1.1)

In [ ]:
EXIT_THR = 0.4

fig, ax1 = plt.subplots(figsize=(30,10))

ax1.plot(pFullsize[:,1], color="gray")

ax2 = ax1.twinx()

y_dir_naned = copy.deepcopy(y_dir)
y_dir_naned[y_dir_naned == 0] = np.nan

entries = np.array([pFullsize[:,1] >= EXIT_THR])[0,:] * tickdata.loc[:,"close"].values
entries[entries == 0] = np.nan

ax2.plot(tickdata.loc[:,"close"].values, color = "black")
ax2.plot(entries, color = "red", marker="X", markersize=10)

ax1.set_ylim(0,1.1)

In [ ]:
ENTRY_THR2 = 0.5
ENTRY_THR2 = 0.0

fig, ax1 = plt.subplots(figsize=(30,10))

ax1.plot(pFullsize[:,1], color="gray")

ax2 = ax1.twinx()

y_dir_naned = copy.deepcopy(y_dir)
y_dir_naned[y_dir_naned == 0] = np.nan

entries = np.array((pFullsize[:,0] >= ENTRY_THR) & (pFullsize[:,1] <= ENTRY_THR2)) * tickdata.loc[:,"close"].values
entries[entries == 0] = np.nan

ax2.plot(tickdata.loc[:,"close"].values, color = "black")
ax2.plot(entries, color = "green", marker="X", markersize=10)

ax1.set_ylim(-1.1,1.1)

In [ ]:
stop

# Generate the X-Frames

In [ ]:
for i in tqdm(range(len(testfiles))):
  # Load the tick table
  tickpath = os.path.join(TEST_PATH, testfiles[i])
  tickdata = pd.read_csv(tickpath)
  tickdata.set_index("startsAt", inplace = True)
  tickdata.sort_index()

  # Print the reuqired batches as info
  required_batches = int(np.floor((tickdata.shape[0]-X_LOOKBACK_CNT) / BATCH_SIZE))
  print(f"Required batches: {required_batches}")

  # Create a X frame generator (Todo: the csv is loaded again)
  xFrameGen = XFrameGenerator(testfiles[i])

  # Generate all frame
  n = 0
  xFrames = None

  while True:
    try:
      print(f"Batch {n} of {required_batches}")
      n += 1

      ne = next(xFrameGen)
      if xFrames is None:
        xFrames = ne
      else:
        xFrames = np.concatenate([xFrames, ne], axis=0)

    except StopIteration:
      break

  xFramePath = tickpath.replace(".csv", "_xFrames.npy")
  np.save(xFramePath, xFrames)

In [ ]:
stop

# Predict on the GPU
# Todo: Load xFrames

In [ ]:
if not os.path.exists(os.path.join(TEST_PATH, "predicted")):
  os.mkdir(os.path.join(TEST_PATH, "predicted"))

SAVE_PATH = os.path.join(TEST_PATH, "predicted", copy_filenames[0].split('/')[-3] + "_" + copy_filenames[0].split('/')[-2])

if not os.path.exists(SAVE_PATH):
  os.mkdir(SAVE_PATH)

SAVE_PATH

In [ ]:
SAVE_PATH

In [ ]:
# Get test file names - npy files
import os
from os import listdir
from os.path import isfile, join

testfiles = [f for f in listdir(TEST_PATH) if isfile(join(TEST_PATH, f)) and ".npy" in f ]
testfiles = sorted(testfiles)
len(testfiles)

In [ ]:
for i in tqdm(range(len(testfiles))):
  print(testfiles[i])

  # Load the tick table
  tickpath = os.path.join(TEST_PATH, testfiles[i].replace("_xFrames.npy", ".csv"))
  tickdata = pd.read_csv(tickpath)
  tickdata.set_index("startsAt", inplace = True)
  tickdata.sort_index()

  # Load the xFrame
  xFrames = np.load(os.path.join(TEST_PATH, testfiles[i]))

  # # Check time dimension
  # if (tickdata.shape[0]-X_LOOKBACK_CNT) != xFrames.shape[0]:
  #   print(f"(tickdata.shape[0]-X_LOOKBACK_CNT) != xFrames.shape[0] for {testfiles[i]}")
  #   print(("tickdata.shape[0]", tickdata.shape[0]))
  #   print(("xFrames.shape[0]", xFrames.shape[0]))
  #   continue

  # Predict
  p = model.predict(xFrames)
  print(p.shape)

  # Adapt prediction to input shape (a window in the beginning cannot be predicted, as lookback into the past is required)
  pFullsize = np.empty((tickdata.shape[0], p.shape[1]))
  pFullsize[:] = np.nan
  pFullsize[-p.shape[0]:,:] = p

  tickdata['p_dir'] = pFullsize[:,0]
  tickdata['p_dir2nd'] = pFullsize[:,1]

  # Drop nan rows
  tickdata.dropna(inplace = True)

  # Save it
  tickdata.to_csv(os.path.join(SAVE_PATH, testfiles[i].replace("_xFrames.npy", ".csv")))

In [ ]:
stop

# Predict all files

In [ ]:
if False:
  for i in tqdm(range(2,len(testfiles))):
    # Load the tick table
    tickpath = os.path.join(TEST_PATH, testfiles[i])
    tickdata = pd.read_csv(tickpath)
    tickdata.set_index("startsAt", inplace = True)
    tickdata.sort_index()

    # Print the reuqired batches as info
    required_batches = int(np.floor((tickdata.shape[0]-X_LOOKBACK_CNT) / BATCH_SIZE))
    print(f"Required batches: {required_batches}")
  
    # Create a X frame generator (Todo: the csv is loaded again)
    xFrameGen = XFrameGenerator(testfiles[i])
    p = model.predict(xFrameGen)
    print(p.shape)

    # Adapt prediction to input shape (a window in the beginning cannot be predicted, as lookback into the past is required)
    pFullsize = np.empty((tickdata.shape[0], p.shape[1]))
    pFullsize[:] = np.nan
    pFullsize[-p.shape[0]:,:] = p

    tickdata['p_dir'] = pFullsize[:,0]
    tickdata['p_dir2nd'] = pFullsize[:,1]

    # Drop nan rows
    tickdata.dropna(inplace = True)

    # Save it
    tickdata.to_csv(os.path.join(SAVE_PATH, testfiles[i]))

In [ ]:
if False:
  for i in tqdm(range(2,len(testfiles))):
    # Load the tick table
    tickpath = os.path.join(TEST_PATH, testfiles[i])
    tickdata = pd.read_csv(tickpath)
    tickdata.set_index("startsAt", inplace = True)
    tickdata.sort_index()

    # Print the reuqired batches as info
    required_batches = int(np.floor((tickdata.shape[0]-X_LOOKBACK_CNT) / BATCH_SIZE))
    print(f"Required batches: {required_batches}")
  
    # Create a X frame generator (Todo: the csv is loaded again)
    xFrameGen = XFrameGenerator(testfiles[i])
    p = model.predict(xFrameGen)
    print(p.shape)

    # Adapt prediction to input shape (a window in the beginning cannot be predicted, as lookback into the past is required)
    pFullsize = np.empty((tickdata.shape[0], p.shape[1]))
    pFullsize[:] = np.nan
    pFullsize[-p.shape[0]:,:] = p

    tickdata['p_dir'] = pFullsize[:,0]
    tickdata['p_dir2nd'] = pFullsize[:,1]

    # Drop nan rows
    tickdata.dropna(inplace = True)

    # Save it
    tickdata.to_csv(os.path.join(SAVE_PATH, testfiles[i]))

# Old stuff

In [ ]:
stop

In [ ]:
def TransformXFrameToSentenceMaskLatest(Xin, hideNonMaskedTokensInY = True):

  total_item_count = BATCH_SIZE * X_LOOKBACK_CNT

  # Store the input shape
  inputShape = Xin.shape

  # Create a "sentence"
  X_normed = NormForBirch(Xin)
  X_normed = np.reshape(X_normed, (X_normed.shape[0] * X_normed.shape[1], X_normed.shape[2]))
  sentence = brc.predict(X_normed)
  sentence = np.reshape(sentence, inputShape[:2])

  # Mask it
  sentenceMasked = copy.deepcopy(sentence) #np.array(np.reshape(sentence, (sentence.shape[0] * sentence.shape[1])))
  sentenceMasked[:,-1] = MASK_TOKEN_ID

  # Replace control tokens
  sentenceMasked[sentenceMasked == BOS_TOKEN_ID] = MASK_TOKEN_ID
  sentenceMasked[sentenceMasked == PAD_TOKEN_ID] = MASK_TOKEN_ID
  sentenceMasked[sentenceMasked == EOS_TOKEN_ID] = MASK_TOKEN_ID

  # sentenceMasked = np.reshape(sentenceMasked, sentence.shape)

  # Replace all token positions in y where no mask is in X with -1 
  if hideNonMaskedTokensInY:
    sentence[MASK_TOKEN_ID != sentenceMasked] = -1

  return (sentence,sentenceMasked)

# Generate an array for back-translating vocab intergers to chart data

In [ ]:
stop
can be skipped to next section

In [ ]:
def backTranslateGenerator():
  it = DataStreamCreator.FileListToDataStream(trainfiles, BATCH_SIZE, TRAIN_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              parallel_generators = 32, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              norm_price_related_indicators = False,
                                              shuffle=True)

  while True:
    try:
      ne = next(it)
      sentence, sentenceMasked = TransformXFrameToSentenceMaskLatest(ne[0], hideNonMaskedTokensInY = False)
      
      yield (sentenceMasked, sentence, ne)

    except StopIteration as si:
      logging.warning("StopIteration in FileListToDataStream")
      logging.warning(si)
      return

btg = backTranslateGenerator()
sentenceMasked, sentence, (XData, yData)  = next(btg)

In [ ]:
btg = backTranslateGenerator()

XFrameDict = {}
for i in range(256):
  XFrameDict[i] = []

for step in tqdm(range(200)):
  sentenceMasked, sentence, (XData, yData)  = next(btg)
  for s in range(sentence.shape[0]):
    # print(("s", s))

    for wordIndex in range(sentence.shape[1]):
      # print(("wordIndex"), wordIndex)

      word = sentence[s, wordIndex]

      # print(word)

      XFrame = XData[s, wordIndex, :]

      XFrameDict[word].append(XFrame)


In [ ]:
for k in XFrameDict.keys():
  print((k, len(XFrameDict[k])))

In [ ]:
np.max(list(XFrameDict.keys()))

In [ ]:
XFrameDict[0][0].shape[0]

In [ ]:
# Calculate the median X frame for every "word"/token
XFrames = np.zeros((
    np.max(list(XFrameDict.keys()))+1,
    XFrameDict[0][0].shape[0]
    ))

for k in XFrameDict.keys():
  XFrames[k,:] = np.clip(np.mean(np.array(XFrameDict[k]), axis=0), -100, 100)

In [ ]:
XFrames.shape

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))
_ = ax1.plot(XFrames.transpose())

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))
_ = ax1.plot(XFrames[0,:])

In [ ]:
todo check if really intended to be saved
np.save("/content/drive/MyDrive/Privat/Crypto/XFrames256.npy", XFrames)

In [ ]:
stop

# Predict stuff

In [ ]:
#@title dummyTrainMLM - DISABLED
# def dummyTrainMLM():
#   it = DataStreamCreator.FileListToDataStream(trainfiles, BATCH_SIZE, TRAIN_PATH,
#                                               smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
#                                               X_lookback_cnt = X_LOOKBACK_CNT,
#                                               y_lookahead_cnt = Y_LOOKAHEAD_CNT,
#                                               gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
#                                               shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
#                                               y_type = Y_TYPE,
#                                               parallel_generators = 8, random_seed=RANDOM_SEED,
#                                               y_exponent = 1.0,
#                                               norm_price_related_indicators = False,
#                                               shuffle=True)

#   while True:
#     try:
#       XOriginal, yOriginal = next(it)
#       # sentence, sentenceMasked = TransformXFrameToMaskedSentence(ne[0])
#       sentence, sentenceMasked = TransformXFrameToSentenceMaskLatest(XOriginal)

#       yield sentenceMasked, sentence, (XOriginal, yOriginal)

#     except StopIteration as si:
#       logging.warning("StopIteration in FileListToDataStream")
#       logging.warning(si)
#       return

# tfgenTrainMLM = tf.data.Dataset.from_generator(dummyTrainMLM, output_types = (tf.int64, tf.int64), output_shapes=((BATCH_SIZE,X_LOOKBACK_CNT), (BATCH_SIZE,X_LOOKBACK_CNT)))
# tfgenTrainMLM = tfgenTrainMLM.prefetch(tf.data.AUTOTUNE)
# tfgenTrainMLM

In [ ]:
def dummyTestMLM():
  it = DataStreamCreator.FileListToDataStream(testfiles, BATCH_SIZE, TEST_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              parallel_generators = 4, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              shuffle=False)
 
  total_item_count = BATCH_SIZE * X_LOOKBACK_CNT
  items_to_mask_count = int(np.round(total_item_count * MASK_FACTOR))

  while True:
    try:
      XOriginal, yOriginal = next(it)
      # sentence, sentenceMasked = TransformXFrameToMaskedSentence(ne[0])
      sentence, sentenceMasked = TransformXFrameToSentenceMaskLatest(XOriginal, hideNonMaskedTokensInY = False)

      yield sentenceMasked, sentence, (XOriginal, yOriginal)

    except StopIteration as si:
      logging.warning("StopIteration in FileListToDataStream")
      logging.warning(si)
      return
  
tfgenTestMLM = dummyTestMLM()
# tfgenTestMLM = tf.data.Dataset.from_generator(dummyTestMLM, output_types = (tf.int32, tf.int32), output_shapes=((BATCH_SIZE,X_LOOKBACK_CNT), (BATCH_SIZE,X_LOOKBACK_CNT)))
# tfgenTestMLM

In [ ]:
wurst = next(tfgenTestMLM)

In [ ]:
wurst[0].shape

In [ ]:
wurst[1].shape

In [ ]:
wurst[0][1,:]

In [ ]:
wurst[1][1,:]

In [ ]:
_ = plt.hist(wurst[0].flatten())

In [ ]:
cfg = RobertaConfig()
cfg.vocab_size = VOCAB_SIZE
cfg.max_position_embeddings = X_LOOKBACK_CNT
cfg.bos_token_id = BOS_TOKEN_ID
cfg.pad_token_id = PAD_TOKEN_ID
cfg.eos_token_id = EOS_TOKEN_ID
cfg

In [ ]:
#@title CreateModelRobertaMLMTraining
def CreateModelRobertaMLMTraining():

  # Build your model input
  inputSentence = Input(shape=(X_LOOKBACK_CNT), name='input', dtype='int32')
  
  nlp = TFRobertaModel(cfg)(inputSentence)

  categories = Dense(1024, activation='softmax', name="Categories")(nlp['last_hidden_state'])

  outputs = [categories]

  mnamesuffix = "_2"

  # And combine it all in a model object
  model = Model(inputs=inputSentence, outputs=outputs, name='ModelRobertaMLMTraining'+mnamesuffix)

  return model

# model = CreateModelRobertaMLMTraining()
# model.summary(line_length=220)

In [ ]:
#@title CreateModelRobertaMLMTrainingDropout
def CreateModelRobertaMLMTrainingDropout():

  # Build your model input
  inputSentence = Input(shape=(X_LOOKBACK_CNT), name='input', dtype='int32')
  
  nlp = TFRobertaModel(cfg, name="Roberta")(inputSentence)

  categories = Dense(VOCAB_SIZE, activation='softmax', name="Categories")(nlp['last_hidden_state'])
  drp = Dropout(0.15, name="CategoriesDropout")(categories)

  # categories_weighted = tf.multiply(drp, CLASS_WEIGHTS_TF_CONST)

  outputs = [drp]

  mnamesuffix = "_1"

  # And combine it all in a model object
  model = Model(inputs=inputSentence, outputs=outputs, name='ModelRobertaMLMTrainingDropout'+mnamesuffix)

  return model

# model = CreateModelRobertaMLMTrainingDropout()
# model.summary(line_length=220)

In [ ]:
#@title CreateModelRobertaLastWordPrediction
def CreateModelRobertaLastWordPrediction():

  # Build your model input
  inputSentence = Input(shape=(X_LOOKBACK_CNT), name='input', dtype='int32')
  
  nlp = TFRobertaModel(cfg, name="Roberta")(inputSentence)

  flat = Flatten(name="FlattenNLP")(nlp['last_hidden_state'])
  lastWord = Dense(VOCAB_SIZE, activation='softmax', name="Categories", dtype=tf.float32)(flat)

  outputs = [lastWord]

  mnamesuffix = "_1"

  # And combine it all in a model object
  model = Model(inputs=inputSentence, outputs=outputs, name='ModelRobertaLastWordPrediction'+mnamesuffix)

  return model

# model = CreateModelRobertaLastWordPrediction()
# model.summary(line_length=220)

In [ ]:
# #@title Load the class weights
# with file_io.FileIO('gs://ticks_with_indicators_with_volume/sentences/set2/weightArray.npy', mode='rb') as input_f:
#   with file_io.FileIO('/content/weightArray.npy', mode='wb+') as output_f:
#     output_f.write(input_f.read())

# CLASS_WEIGHTS = np.empty((1,1,1024))
# CLASS_WEIGHTS[0,0,:] = np.load('/content/weightArray.npy')

In [ ]:
# plt.plot(CLASS_WEIGHTS[0,0,:])
# plt.yscale("log")

In [ ]:
# CLASS_WEIGHTS[:] = 1.0

In [ ]:
# CLASS_WEIGHTS_TF_CONST = tf.constant(tf.convert_to_tensor(CLASS_WEIGHTS,dtype=tf.float32))

In [ ]:
#@title CreateModelRobertaMLMTrainingWeighted
def CreateModelRobertaMLMTrainingWeighted():

  # Build your model input
  inputSentence = Input(shape=(X_LOOKBACK_CNT), name='input', dtype='int32')
  
  nlp = TFRobertaModel(cfg)(inputSentence)

  categories = Dense(1024, activation='softmax', name="Categories")(nlp['last_hidden_state'])
  categories_weighted = tf.multiply(categories, CLASS_WEIGHTS_TF_CONST)

  outputs = [categories_weighted]

  mnamesuffix = "_1"

  # And combine it all in a model object
  model = Model(inputs=inputSentence, outputs=outputs, name='ModelRobertaMLMTrainingWeighted'+mnamesuffix)

  return model

# model = CreateModelRobertaMLMTrainingWeighted()
# model.summary(line_length=220)

In [ ]:
#@title CreateModelDummy
def CreateModelDummy():

  # Build your model input
  input = Input(shape=(X_LOOKBACK_CNT,FEATURES), name='input', dtype='float32')
  
  d = Dense(32)(Flatten()(input))
  d = Dense(16)(d)
  d = Dense(16)(d)
  
  classifier = Dense(512, activation='relu')(d)
  output = Dense(3, activation='softmax')(classifier)
  outputs = [ev]

  # And combine it all in a model object
  model = Model(inputs=input, outputs=outputs, name='RelativeLSTMDenseReduced')

  return model
# m = CreateModelDummy()
# m.summary()

# Predict

In [ ]:
model = CreateModelRobertaLastWordPrediction()

In [ ]:
model.summary(line_length=220)

In [ ]:
if not os.path.exists("/content/model.h5"):
  copy_filenames = ['gs://ticks_with_indicators_with_volume/chk/TPU_ModelRobertaLastWordPrediction_1_128LB_256VC_FullPrediction/cp_daily_valid_49_end/model.h5'
                    ]

  for p in copy_filenames:
    fn = p.split("/")[-1]

    with file_io.FileIO(p, mode='rb') as input_f:
      with file_io.FileIO(os.path.join("/content", fn), mode='wb+') as output_f:
        output_f.write(input_f.read())
        print("Pulled from bucket: '" + fn + "'")
else:
  print("DID NOT COPY FILE")

In [ ]:
model.load_weights("/content/model.h5")

In [ ]:
sentenceMasked, sentence, (XOriginal, yOriginal) = next(tfgenTestMLM)

In [ ]:
p = model.predict(sentenceMasked)
p.shape

In [ ]:
# p_backnormed = np.squeeze(np.nan_to_num((p / CLASS_WEIGHTS), nan=0, posinf=0, neginf=0))
# p_backnormed.shape

#p_backnormed = p

In [ ]:
p.shape

In [ ]:
_ = plt.plot(p[:,:].transpose())

In [ ]:
pcopy = copy.deepcopy(p)
def func(x):
    x[np.argwhere(x != x.max())] = 0
    x[x != 0] = 1
    x *= np.array(range(256))
    return x

pmax = np.apply_along_axis(func, 1, pcopy)

In [ ]:
pmax.shape

In [ ]:
_ = plt.plot(pmax[:,:].transpose())

In [ ]:
pSentence = np.sum(pmax, axis=1)
pSentence.shape

In [ ]:
_ = plt.plot(pSentence)

In [ ]:
# for i in range(64):
#   print(np.var(pSentence[i,:]))

In [ ]:
showIndex = 9

In [ ]:
sentenceMasked[showIndex,:]

In [ ]:
sentence[showIndex,:]

In [ ]:
pSentence[showIndex,:]

In [ ]:
sentenceMasked[showIndex,:] - pSentence[showIndex,:]

In [ ]:
sentence[showIndex,:]

In [ ]:
_ = plt.plot(pmax[0,:,:])

## Convert it back to an time-based XFrame

In [ ]:
XFrameTemplates = np.load("/content/drive/MyDrive/Privat/Crypto/XFrames256.npy")
XFrameTemplates.shape

In [ ]:
XFrame = np.zeros((pSentence.shape[1],XFrameTemplates.shape[1]))

for i, word in enumerate(pSentence[showIndex,:]):
  XFrame[i,:] = XFrameTemplates[int(word), :]

In [ ]:
pSentence[showIndex,:]

In [ ]:
sentence[showIndex,:]

In [ ]:
# From source data
XFrameGroundTruth = np.zeros((sentence.shape[1],XFrameTemplates.shape[1]))

for i, word in enumerate(sentence[showIndex,:]):
  XFrameGroundTruth[i,:] = XFrameTemplates[int(word), :]

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))
_ = ax1.plot(XFrame)

In [ ]:
XFrame.shape

In [ ]:
XFrameGroundTruth[:,3]

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))
_ = ax1.plot(XFrame[:,3])
# _ = ax1.plot(XFrameGroundTruth[:,3], color="green")
_ = ax1.plot(XOriginal[showIndex,:,3], color="green")

# ax1.set_ylim(0,2)

In [ ]:
.shape

In [ ]:
pSentence.shape

In [ ]:
stop

In [ ]:
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-04,
    epsilon=1e-06,
    beta_1=0.9,
    beta_2=0.98,
    decay=0.01)

In [ ]:
# CLASS_WEIGHTS = np.empty((1,1,1024))

# for i in range(1024):
#   CLASS_WEIGHTS[0,0,i] = i * 1.0

In [ ]:
# Create a masked loss to predict only the missing tokens
# https://stackoverflow.com/questions/56328140/how-do-i-implement-a-masked-softmax-cross-entropy-loss-function-in-keras

SCCE = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

def sparse_crossentropy_masked_weighted(y_true, y_pred):
  y_true_masked = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))

  y_pred_weighted = tf.multiply(y_pred, CLASS_WEIGHTS)
  y_pred_masked = tf.boolean_mask(y_pred_weighted, tf.not_equal(y_true, -1))

  return tf.reduce_sum(SCCE(y_true_masked, y_pred_masked)) * (1. / BATCH_SIZE)

In [ ]:
# Compile the model
# model.compile(
#     optimizer = optimizer,
#     loss = [tf.keras.losses.MeanSquaredError()], 
#     metrics=[tf.keras.losses.MeanAbsoluteError(), tf.keras.losses.MeanAbsolutePercentageError()])
model.compile(
    optimizer = optimizer,
    loss = sparse_crossentropy_masked_weighted, 
    metrics=None)
# model.compile(
#     optimizer = optimizer,
#     loss = ['mse'], 
#     metrics=['mae'])

In [ ]:
model.fit(tfgenTrainMLM)

# Calculate class weights

In [ ]:
def genCalcWeights():
  it = DataStreamCreator.FileListToDataStream(trainfiles, BATCH_SIZE, TRAIN_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              parallel_generators = 8, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              norm_price_related_indicators = False,
                                              shuffle=True)

  while True:
    try:
      ne = next(it)
      sentence, sentenceMasked = TransformXFrameToMaskedSentence(ne[0], hideNonMaskedTokensInY=False)

      yield (sentenceMasked, sentence)

    except StopIteration as si:
      logging.warning("StopIteration in FileListToDataStream")
      logging.warning(si)
      return

gcw = genCalcWeights()

In [ ]:
countArray = np.zeros((1024))

for ep in tqdm(range(1000)):
  maskedSentence, fullSentence = next(gcw)

  for w in fullSentence:
    countArray[w] += 1

In [ ]:
plt.plot(countArray)

In [ ]:
normedArray = countArray / np.sum(countArray)
plt.plot(normedArray)

In [ ]:
weightArray = np.array(normedArray)

maskArray = np.empty(weightArray.shape)
maskArray[:] = 1.0
maskArray[weightArray == 0.0] = 0.0

weightArray = 1.0 / (normedArray)
weightArray = np.nan_to_num(weightArray,nan=0,posinf=0,neginf=0)

weightArray /= np.sum(weightArray)
weightArray[weightArray >= 0.01] = 0.01
weightArray /= np.sum(weightArray)

plt.plot(weightArray)

In [ ]:
plt.plot(weightArray)
plt.yscale("log")

In [ ]:
np.sum(weightArray)

In [ ]:
plt.plot(countArray * weightArray)

In [ ]:
# Save the weight array

In [ ]:
np.save("/content/weightArray.npy", weightArray)

In [ ]:
for fn in ["/content/weightArray.npy"]:
  fnBucket = fn.replace('/content', 'gs://ticks_with_indicators_with_volume/sentences/set2')

  with file_io.FileIO(fn, mode='rb') as input_f:
    with file_io.FileIO(fnBucket, mode='wb+') as output_f:
      output_f.write(input_f.read())
  print("Pushed to bucket: '" + fnBucket + "'")

In [ ]:
#@title Load the class weights
with file_io.FileIO('gs://ticks_with_indicators_with_volume/sentences/set2/weightArray.npy', mode='rb') as input_f:
  with file_io.FileIO('/content/weightArray.npy', mode='wb+') as output_f:
    output_f.write(input_f.read())

CLASS_WEIGHTS = np.empty((1,1,1024))
CLASS_WEIGHTS[0,0,:] = np.load('/content/weightArray.npy')

In [ ]:
stop

In [ ]:
from sklearn.cluster import Birch

In [ ]:
brc = Birch(n_clusters=128)

In [ ]:
def NormForBirch(XIn):
  normed = XIn
  normed[normed >= 100] = 100
  normed[normed <= -100] = -100
  normed += 100 + 1
  normed = np.log10(normed)
  normed -= np.log10(100 + 1)
  
  return normed

In [ ]:
# Load the BIRCH categorizer
with open("/content/drive/MyDrive/Privat/Crypto/birch.pkl", 'rb') as pickle_file:
  brc = pickle.load(pickle_file)

In [ ]:
brc.transform

In [ ]:
np.save("/content/drive/MyDrive/diff.csv", diff)

In [ ]:
#plt.plot(diff[0,:])
plt.plot(fullSentence[0,:])
plt.plot(maskedSentence[0,:])

In [ ]:
stop

# Train the model

In [ ]:
CHKPNT_NAME = "{}_T{}_{}LB_LF{}_EN{}_{}_{}_EX{}_{}_SM{}_{}".format(model.name, Y_TYPE, X_LOOKBACK_CNT, EXPECTED_GAIN_LOOKFORWARD, ENTR_THR, ENTR_THR2, ENTR_THR3, EXIT_THR, EXIT_THR2, SMCNT, SMCNT2)
CHKPNT_NAME

# Train


#Pretraining

https://huggingface.co/roberta-base

The model was trained on 1024 V100 GPUs for 500K steps with a batch size of 8K and a sequence length of 512. The optimizer used is Adam with a learning rate of 6e-4, β1=0.9\beta_{1} = 0.9β1​=0.9, β2=0.98\beta_{2} = 0.98β2​=0.98 and ϵ=1e−6\epsilon = 1e-6ϵ=1e−6, a weight decay of 0.01, learning rate warmup for 24,000 steps and linear decay of the learning rate after.

In [ ]:
# Set an optimizer
optimizer = Adam(
    learning_rate=1e-06,
    epsilon=1e-06,
    beta_1=0.9,
    beta_2=0.98,
    decay=0.01)

In [ ]:
# Compile the model
# model.compile(
#     optimizer = optimizer,
#     loss = [tf.keras.losses.MeanSquaredError()], 
#     metrics=[tf.keras.losses.MeanAbsoluteError(), tf.keras.losses.MeanAbsolutePercentageError()])
model.compile(
    optimizer = optimizer,
    loss = [tf.keras.losses.SparseCategoricalCrossentropy()], 
    metrics=None)
# model.compile(
#     optimizer = optimizer,
#     loss = ['mse'], 
#     metrics=['mae'])

In [ ]:
# # Load the input Conv weights (cannot be trained due to float-->int cast)
# copy_filenames = ['gs://ticks_with_indicators_with_volume/chk/MergeFeatureToOneFloat_1_T3_128LB_LF168_EN0.9_0.8_0.1_EX-0.75_-0.7_SM48_48/cp_daily_valid_0_00000/ModelMergeFeatureToOneFloat.h5']

# for p in copy_filenames:
#   fn = p.split("/")[-1]

#   with file_io.FileIO(p, mode='rb') as input_f:
#     with file_io.FileIO(os.path.join("/content", fn), mode='wb+') as output_f:
#       output_f.write(input_f.read())
#       print("Pulled from bucket: '" + fn + "'")

# mInit = CreateModelMergeFeatureToOneFloat()

# # # Saved model to: 'gs://ticks_with_indicators_with_volume/chk/MergeFeatureToOneFloat_1_T3_128LB_LF168_EN0.9_0.8_0.1_EX-0.75_-0.7_SM48_48/cp_daily_valid_0_00000/ModelMergeFeatureToOneFloat.h5'
# mInit.load_weights('/content/ModelMergeFeatureToOneFloat.h5')

# # model.get_layer("MergeFeatureToOneFloat").set_weights(
# #     model.get_layer("MergeFeatureToOneFloat").get_weights()
# # )

In [ ]:
# mInit.summary()

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# # !gcloud config set project {project_id} -y
# !gsutil cp -R gs://ticks_with_indicators_with_volume/logs/ModelNLP_1_T3_128LB_LF168_EN0.9_0.8_0.1_EX-0.75_-0.7_SM48_4820220915-162219 /content/logs

In [ ]:
# %tensorboard --logdir /content/logs

In [ ]:
# from tensorboard import notebook
# notebook.list() # View open TensorBoard instances

In [ ]:
model.summary(line_length=220)

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
  def __init__(self, save_freq, val_freq, checkpoint_path, model_name, epoch_add=0):
    self.save_freq = save_freq
    self.val_freq = val_freq
    self.checkpoint_path = checkpoint_path
    self.model_name = model_name
    self.current_epoch = 0
    self.epoch_add = epoch_add

  def on_epoch_begin(self, epoch, logs=None):
    self.current_epoch = epoch + self.epoch_add
    # keys = list(logs.keys())
    # print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

  def on_train_epoch_end(self, logs=None):
    self.saveTheModel(99999,logs)

  def on_train_batch_end(self, batch, logs=None):
    self.saveTheModel(batch,logs)

  def saveTheModel(self, batch, logs=None):
    if 0 < batch and 0 == batch % self.save_freq:
      _save_folder = os.path.join(self.checkpoint_path,
                                self.model_name,
                                "cp_daily_valid_{:01d}_{:05d}".format(self.current_epoch, batch)
                                )

      _model_path_local = os.path.join("/content/", "model.h5")
      _model_path_bucket = os.path.join(_save_folder, "model.h5")

      model.save(_model_path_local)
     
      # Copy model.h5 over to Google Cloud Storage
      with file_io.FileIO(_model_path_local, mode='rb') as input_f:
          with file_io.FileIO(_model_path_bucket, mode='wb+') as output_f:
              output_f.write(input_f.read())
              print("\nSaved model to: '" + _model_path_bucket + "'")

      # Save optimizer config
      c = copy.deepcopy(self.model.optimizer.get_config())

      fp = os.path.join(_save_folder, "c.pickle")
      with file_io.FileIO(fp, mode='wb+') as handle:
        pickle.dump(c, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Saved optimizer config to: '" + fp + "'")

      # Save optimizer weights
      w = copy.deepcopy(self.model.optimizer.get_weights())

      fp = os.path.join(_save_folder, "w.pickle")
      with file_io.FileIO(fp, mode='wb+') as handle:
        pickle.dump(w, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Saved optimizer weights to: '" + fp + "'")

    if 0 < batch and 0 == batch % self.val_freq:
      print("-------------------------EVAL-------------------------")
      model.evaluate(tfgenTestMLM)
      print("\n-------------------------EVAL-------------------------")

In [ ]:
epoch_add = 0

In [ ]:
CALLBACK_EVERY_N_BATCHES = 2000

In [ ]:
cc = CustomCallback(checkpoint_path = checkpoint_path,
                    model_name = CHKPNT_NAME,
                    save_freq = CALLBACK_EVERY_N_BATCHES,
                    val_freq = CALLBACK_EVERY_N_BATCHES,
                    epoch_add = epoch_add)

In [ ]:
import datetime
log_dir = "gs://ticks_with_indicators_with_volume/logs/" + CHKPNT_NAME + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
     histogram_freq=0,
     update_freq=CALLBACK_EVERY_N_BATCHES 
     )

In [ ]:
# copy_filenames = ['gs://ticks_with_indicators_with_volume/chk/ModelNLP_1_T3_128LB_LF168_EN0.9_0.8_0.1_EX-0.75_-0.7_SM48_48/cp_daily_valid_10_16000/model.h5'
#                   ]

# for p in copy_filenames:
#   fn = p.split("/")[-1]

#   with file_io.FileIO(p, mode='rb') as input_f:
#     with file_io.FileIO(os.path.join("/content", fn), mode='wb+') as output_f:
#       output_f.write(input_f.read())
#       print("Pulled from bucket: '" + fn + "'")

In [ ]:
# model.load_weights("/content/model.h5")

In [ ]:
# modelFirstLayers = CreateModelViTPara4TimeAndFeatureTo2D()

In [ ]:
# modelFirstLayers.load_weights("/content/model.h5")

In [ ]:
# for l in modelFirstLayers.layers:
#   if 0 < len(l.get_weights()):
#     lname = l.name
#     lweights = l.get_weights()

#     # Bugfix:
#     if "tf_vi_t_for_image_classification_1" == lname:
#       lname = "tf_vi_t_for_image_classification"

#     try:
#       model.get_layer(lname).set_weights(lweights)
#       # model.get_layer(lname).trainable = False

#       print("Applied " + l.name)
#     except ValueError as ve:
#       print("Error")
#       print(ve)

In [ ]:
# with open("/content/w.pickle", 'rb') as pickle_file:
#   w = pickle.load(pickle_file)
# model.optimizer.set_weights(w)

In [ ]:
# class_weights = {0:2.0, 1: 2.0, 2: 1.0}

In [ ]:
# model.optimizer.learning_rate = 1e-5

In [ ]:
model.fit(tfgenTrainMLM, epochs=200, verbose = 1, callbacks=[tensorboard_callback, cc], validation_data=tfgenTestMLM, validation_freq=500, class_weight=CLASS_WEIGHT)

In [ ]:
p = model.predict(tfgenTest)
p.shape

In [ ]:
p[1].shape

In [ ]:
plt.plot(p[1][0,:])

In [ ]:
p